# Alumnos:
Luis Felipe Acosta Zavaleta (20143350)

Raul Emerson Amao Guerra (20180592)

Manuel Antonio Gil Soto Pisco (20098125)

Ruth Marina Chavez Pacheco (20161350)

Alejandro Mosquera Ospina (20125293)

In [7]:
#Libreries

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

### My part_Raúl

In [ ]:
# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
tables_list = []

# We are selecting the last 5 elections
for k in range(21, 26):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)
    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(k):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    tables_list.append(table_data)

    time.sleep(2)            

    print(f"Data for election number {k}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
final_table = pd.concat(tables_list, ignore_index=True)

# Save the final table to an Excel file
final_table.to_excel('Elecciones_Data.xlsx', index=False)

### Ruth: elections 1956-1985

In [9]:
driver = webdriver.Chrome()

driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")
url = 'https://infogob.jne.gob.pe/Eleccion'

elecciones_1985_1956 = []

# Select 5 years from 1956 to 1985
for x in range(16, 21):
    driver.get(url)
    time.sleep(3)
    proceso = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div").click()
    time.sleep(1)
    elecciones = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]").click()
    time.sleep(1)
    presidenciales = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div").click()
    time.sleep(1)
    for _ in range(x):
        presidenciales.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5) 
        
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2) 
        
    verdatos = driver.find_element(By.XPATH,"/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button").click() # Select "Ver datos de elección"
    time.sleep(1)
    candidatosresultados = driver.find_element(By.XPATH,"/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]").click() # Select "Candidatos y resultados"
    time.sleep(1)

    tabla = driver.find_element(By.ID, 'CandidatosResultados').get_attribute('innerHTML')
    df = pd.read_html(tabla)
    df['Elección'] = eleccion
    
    df = df[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elecciones_1985_1956.append(df)
    time.sleep(2)            

    print(f"Data for election number {k}:\n", df)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
tabla_1985_1956 = pd.concat(elecciones_1985_1956, ignore_index=True)

# Save the final table to an Excel file
tabla_1985_1956.to_excel('Elecciones_1985_1956.xlsx', index=False)

NoSuchDriverException: Message: Unable to obtain chromedriver-win64/chromedriver.exe using Selenium Manager; Message: Unsuccessful command executed: C:\Users\HP\anaconda3\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --output json.
The chromedriver version cannot be discovered
; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


### Part Luis Felipe - Elections 1990 (1st round), 1990 (2nd round), 1995, 2000 (1st round) & 2000 (2nd round)

In [12]:
# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
elections_1990_2000 = []

# We are selecting the last 5 elections
for k in range(11, 15):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)
    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(k):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elections_1990_2000.append(table_data)

    time.sleep(2)            

    print(f"Data for election number {k}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
table_1990_2000 = pd.concat(elections_1990_2000, ignore_index=True)

# Save the final table to an Excel file
table_1990_2000.to_excel('Elecciones_Data.xlsx', index=False)

Data for election number 11:
             Elección               ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2001                        PERU POSIBLE      3871167
1  PRESIDENCIAL 2001             PARTIDO APRISTA PERUANO      2732857
2  PRESIDENCIAL 2001   ALIANZA ELECTORAL UNIDAD NACIONAL      2576653
3  PRESIDENCIAL 2001    FRENTE INDEPENDIENTE MORALIZADOR      1044207
4  PRESIDENCIAL 2001  ALIANZA ELECTORAL SOLUCION POPULAR       179243
5  PRESIDENCIAL 2001         PARTIDO RENACIMIENTO ANDINO        85436
6  PRESIDENCIAL 2001               PARTIDO PROYECTO PAIS        79077
7  PRESIDENCIAL 2001               TODOS POR LA VICTORIA        33080
8  PRESIDENCIAL 2001                     VOTOS EN BLANCO      1260193
9  PRESIDENCIAL 2001                         VOTOS NULOS       402436


Data for election number 12:
             Elección                       ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2000                 ALIANZA ELECTORAL PERU 2000      5528394
1  PRESIDENC

In [13]:
## Finally, we can see the required table
table_1990_2000

,Elección,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 2001,PERU POSIBLE,3871167
1,PRESIDENCIAL 2001,PARTIDO APRISTA PERUANO,2732857
2,PRESIDENCIAL 2001,ALIANZA ELECTORAL UNIDAD NACIONAL,2576653
3,PRESIDENCIAL 2001,FRENTE INDEPENDIENTE MORALIZADOR,1044207
4,PRESIDENCIAL 2001,ALIANZA ELECTORAL SOLUCION POPULAR,179243
5,PRESIDENCIAL 2001,PARTIDO RENACIMIENTO ANDINO,85436
6,PRESIDENCIAL 2001,PARTIDO PROYECTO PAIS,79077
7,PRESIDENCIAL 2001,TODOS POR LA VICTORIA,33080
8,PRESIDENCIAL 2001,VOTOS EN BLANCO,1260193
9,PRESIDENCIAL 2001,VOTOS NULOS,402436
